In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Suppress the warning for this file
# ok, this is likely a bad idea, but there's going to be a lot of these
pd.options.mode.chained_assignment = None

In [3]:
nyt = pd.read_csv("nyt_full.tsv", sep = '\t')
nyt.head()

,year,week,rank,title_id,title,author
0,1931,1931-10-12,1,6477,THE TEN COMMANDMENTS,Warwick Deeping
1,1931,1931-10-12,2,1808,FINCHE'S FORTUNE,Mazo de la Roche
2,1931,1931-10-12,3,5304,THE GOOD EARTH,Pearl S. Buck
3,1931,1931-10-12,4,4038,SHADOWS ON THE ROCK,Willa Cather
4,1931,1931-10-12,5,3946,SCARMOUCHE THE KING MAKER,Rafael Sabatini


In [4]:
nyt.author.nunique()

2210

In [5]:
print(nyt.author)

0                        Warwick Deeping
1                       Mazo de la Roche
2                          Pearl S. Buck
3                           Willa Cather
4                        Rafael Sabatini
                      ...               
60381                             Halsey
60382                       Brit Bennett
60383                        Delia Owens
60384                    Fredrik Backman
60385    Clive Cussler and Boyd Morrison
Name: author, Length: 60386, dtype: object


In [6]:
# replace written & illustrated by 
nyt.author = nyt.author.replace("written and illustrated by ","", regex = True)

## separate complicated entries

In [8]:
#if there is a preposition, there's complication
nyt.author = nyt.author.replace(" And "," and ", regex = True)
nyt['problem'] = nyt.author.str.contains(' and | with | by ', regex = True, case = False)
print("how many problem entries", nyt.problem.sum())
print(nyt.problem.sum()/len(nyt))

# new dataframe with just problems
nyt_review = nyt[nyt.problem == True]
print("how many unique problems", len(nyt_review.author.unique()))

#new dataframe without problems 
nyt_single = nyt[nyt.problem == False]
nyt_single.to_csv('nyt_single_authors.csv', index=False)

how many problem entries 2416
0.04000927367270559
how many unique problems 189


In [9]:
# if there's an author with editor, remove editor, else treat editor as author 
nyt_review.author = nyt_review.author.replace(", edited by ",". Edited by ", regex = True)

#remove 'edited by ' and transfer rest of string to the 'author_alt' column
def resolve_editors(name):
    if 'dited ' in name:
        if name[:10] == 'edited by ':
            return name[10:]
        elif name[:10] != 'edited by ':
            substring = ('. Edited by ')
            return name.split(substring)[0]
    else: return name

nyt_review['author_alt'] = nyt_review.apply(lambda x: resolve_editors(x['author']), axis = 1)

In [10]:
print(nyt_review['author_alt'].unique()[:5])

['Charles Nordhoff and James N. Hall' 'Gideon Wyck'
 'Charles Nordloff and James Norman Hall'
 'Dennis Wheatley and J. H. Links' 'Somerset Maugham']


In [11]:
def resolve_illustrators(name):
    substring = ('. Illustrated by ')
    return name.split(substring)[0]
    
nyt_review['author_alt'] = nyt_review.apply(lambda x: resolve_illustrators(x['author_alt']), axis = 1)
nyt_review.author_alt.unique()[5:25]

array(['Charles Nordhoff and James Norman Hall', 'Armin L. Robinson',
       'Odell Shepard and Willard Shepard', '? by Isabel Bolton',
       'Benedict Freedman and Nancy Freedman',
       'Joseph Auslander and Audrey Wurdemann',
       'James Street and James Childers',
       'Charmian Clift and George Johnston',
       'Harnett T. Kane and Victor Leclerc', 'Marrijane and Joseph Hayes',
       'Dorothy Erskine and Patrick Dennis',
       'William J. Lederer and Eugene Burdick',
       'Arthur Quiller-Couch and Daphne du Maurier',
       'Fletcher Knebel and Charles W. Bailey II',
       'Eugene Burdick and Harvey Wheeler',
       'Terry Southern and Mason Hoffenberg', 'Nicholas Meyer',
       'J.D. Gilman and John Clive', 'Marvin Kalb and Ted Koppel',
       'General Sir John Hackett and Other Top-ranking NATO Generals and Advisors'],
      dtype=object)

In [12]:
#some custom cleaning
#typo in name
nyt_review.author_alt = nyt_review.author_alt.replace("Charles Nordloff","Charles Nordhoff", regex = True)
nyt_review.author_alt = nyt_review.author_alt.replace("William Shatner with Judith$","William Shatner with Judith Reeves-Stevens", regex = True)
nyt_review.author_alt = nyt_review.author_alt.replace("Constantini","Costantini", regex = True)
nyt_review.author_alt = nyt_review.author_alt.replace("J. T. Ellison","J.T. Ellison", regex = True)


In [13]:
def split_two_authors(name):
    substring1 = (' and ')
    substring2 = (' with ')
    if substring1 in name:
        first_author = name.split(substring1)[0]
        second_author = name.split(substring1)[1]
    elif substring2 in name:
        first_author = name.split(substring2)[0]
        second_author = name.split(substring2)[1]
    else:
        first_author = name
        second_author = np.nan
    return first_author, second_author

nyt_review[['author_1', 'author_2']] = nyt_review.apply(lambda x: split_two_authors(x['author_alt']), axis=1, result_type='expand')

In [14]:
nyt_review.author_1.unique()

array(['Charles Nordhoff', 'Gideon Wyck', 'Dennis Wheatley',
       'Somerset Maugham', 'Armin L. Robinson', 'Odell Shepard',
       '? by Isabel Bolton', 'Benedict Freedman', 'Joseph Auslander',
       'James Street', 'Charmian Clift', 'Harnett T. Kane', 'Marrijane',
       'Dorothy Erskine', 'William J. Lederer', 'Arthur Quiller-Couch',
       'Fletcher Knebel', 'Eugene Burdick', 'Terry Southern',
       'Nicholas Meyer', 'J.D. Gilman', 'Marvin Kalb',
       'General Sir John Hackett', 'Arnaud de Borchgrave',
       'Larry Collins', 'Helen Van Slyke', 'adapted by Joan D. Vinge',
       'created by Bill Adler', 'Robert Moss', 'Whitley Strieber',
       'Stephen King', 'Larry Niven', 'Patti Davis', '! by Terry Brooks',
       'Walter J. Boyne', 'compiled by Mary S. Lovell',
       'Raymond Chandler', 'Judith', 'Barry Lopez', 'Michael Dorris',
       'Margaret Weis', 'Arthur C. Clarke', 'Janet', '? by Jimmy Buffett',
       'adapted by Don Ferguson', 'Richard Marcinko', 'David',
       

In [15]:
#replace lost characters
nyt_review.author_1 = nyt_review.author_1.replace("! by ","", regex = True)
nyt_review.author_1 = nyt_review.author_1.replace(r"\? by ","", regex = True)

#replace other ways of saying editor
nyt_review.author_1 = nyt_review.author_1.replace("selected by ","", regex = True)
nyt_review.author_1 = nyt_review.author_1.replace("compiled by ","", regex = True)
nyt_review.author_1 = nyt_review.author_1.replace("translated by ","", regex = True)
nyt_review.author_1 = nyt_review.author_1.replace("adapted by ","", regex = True)
nyt_review.author_1 = nyt_review.author_1.replace("created by ","", regex = True)
#
nyt_review.author_1 = nyt_review.author_1.replace(". Completed by Carol Gino","", regex = True)
nyt_review.author_1 = nyt_review.author_1.replace("William Shatner with Judith Reeves-Stevens","William Shatner", regex = True)
nyt_review.author_1 = nyt_review.author_1.replace("William Shatner with Judith","William Shatner", regex = True)

In [16]:
nyt_review.author_1.unique()

array(['Charles Nordhoff', 'Gideon Wyck', 'Dennis Wheatley',
       'Somerset Maugham', 'Armin L. Robinson', 'Odell Shepard',
       'Isabel Bolton', 'Benedict Freedman', 'Joseph Auslander',
       'James Street', 'Charmian Clift', 'Harnett T. Kane', 'Marrijane',
       'Dorothy Erskine', 'William J. Lederer', 'Arthur Quiller-Couch',
       'Fletcher Knebel', 'Eugene Burdick', 'Terry Southern',
       'Nicholas Meyer', 'J.D. Gilman', 'Marvin Kalb',
       'General Sir John Hackett', 'Arnaud de Borchgrave',
       'Larry Collins', 'Helen Van Slyke', 'Joan D. Vinge', 'Bill Adler',
       'Robert Moss', 'Whitley Strieber', 'Stephen King', 'Larry Niven',
       'Patti Davis', 'Terry Brooks', 'Walter J. Boyne', 'Mary S. Lovell',
       'Raymond Chandler', 'Judith', 'Barry Lopez', 'Michael Dorris',
       'Margaret Weis', 'Arthur C. Clarke', 'Janet', 'Jimmy Buffett',
       'Don Ferguson', 'Richard Marcinko', 'David', 'William J. Bennett',
       'Tom Hegg', 'Alex Walsh', 'William Shatner', 

In [17]:
nyt_review.author_2.unique()[:10]

array(['James N. Hall', nan, 'James Norman Hall', 'J. H. Links',
       'Willard Shepard', 'Nancy Freedman', 'Audrey Wurdemann',
       'James Childers', 'George Johnston', 'Victor Leclerc'],
      dtype=object)

In [18]:
# if there's only one name in the author_1 column, return that row
nyt_review[~nyt_review.author_1.str.contains(' ')]

,year,week,rank,title_id,title,author,problem,author_alt,author_1,author_2
13772,1957,1957-02-24,15,792,BON VOYAGE,Marrijane and Joseph Hayes,True,Marrijane and Joseph Hayes,Marrijane,Joseph Hayes
36330,1990,1990-09-16,10,3609,PRIME DIRECTIVE,Judith and Garfield Reeves-Stevens,True,Judith and Garfield Reeves-Stevens,Judith,Garfield Reeves-Stevens
36346,1990,1990-09-23,11,3609,PRIME DIRECTIVE,Judith and Garfield Reeves-Stevens,True,Judith and Garfield Reeves-Stevens,Judith,Garfield Reeves-Stevens
36361,1990,1990-09-30,11,3609,PRIME DIRECTIVE,Judith and Garfield Reeves-Stevens,True,Judith and Garfield Reeves-Stevens,Judith,Garfield Reeves-Stevens
36378,1990,1990-10-07,13,3609,PRIME DIRECTIVE,Judith and Garfield Reeves-Stevens,True,Judith and Garfield Reeves-Stevens,Judith,Garfield Reeves-Stevens
36395,1990,1990-10-14,15,3609,PRIME DIRECTIVE,Judith and Garfield Reeves-Stevens,True,Judith and Garfield Reeves-Stevens,Judith,Garfield Reeves-Stevens
37334,1991,1991-12-22,5,5511,THE JOLLY CHRISTMAS POSTMAN,Janet and Allan Ahlberg,True,Janet and Allan Ahlberg,Janet,Allan Ahlberg
37350,1991,1991-12-29,6,5511,THE JOLLY CHRISTMAS POSTMAN,Janet and Allan Ahlberg,True,Janet and Allan Ahlberg,Janet,Allan Ahlberg
37367,1992,1992-01-05,8,5511,THE JOLLY CHRISTMAS POSTMAN,Janet and Allan Ahlberg,True,Janet and Allan Ahlberg,Janet,Allan Ahlberg
37383,1992,1992-01-12,9,5511,THE JOLLY CHRISTMAS POSTMAN,Janet and Allan Ahlberg,True,Janet and Allan Ahlberg,Janet,Allan Ahlberg


In [19]:
#individually reunite the single names with their last names
nyt_review.author_1 = nyt_review.author_1.replace("Judith","Judith Reeves-Stevens", regex = True)
nyt_review.author_1 = nyt_review.author_1.replace("Janet","Janet Ahlberg", regex = True)
nyt_review.author_1 = nyt_review.author_1.replace("David","David Eddings", regex = True)
nyt_review.author_1 = nyt_review.author_1.replace("Faye","Faye Kellerman", regex = True)
nyt_review.author_1 = nyt_review.author_1.replace("Marrijane","Marrijane Hayes", regex = True)

nyt_review.author_1.unique()[:5]

array(['Charles Nordhoff', 'Gideon Wyck', 'Dennis Wheatley',
       'Somerset Maugham', 'Armin L. Robinson'], dtype=object)

In [20]:
nyt_single = pd.read_csv("nyt_single_author_demographics.csv")

nyt_single.rename(columns = {'Unnamed: 0':'author'}, inplace=True)
nyt_single.tail()

,author,birth,death,race_eth,education,institution
2015,Lana Del Rey,1985,N/A,White,None,None
2016,Matt Haig,1975,N/A,White,Bachelor's Degree,University of Kent
2017,Rumaan Alam,1977,N/A,Pakistani-American,Bachelor’s Degree,University of Michigan
2018,Susie Yang,1983,N/A,Asian American,MFA,University of California
2019,Halsey,1994,N/A,White,Bachelor's Degree,Rhode Island School of Design


![join](IMG_0722.jpg)

In [22]:
# return all of the names in nyt_review.author_1 that are NOT in nyt_single.author
# going to use an outer merge

nyt_review_not_in_single = nyt_single.merge(nyt_review, how='outer', left_on = 'author', right_on = 'author_1', indicator = True)
nyt_review_not_in_single =nyt_review_not_in_single[nyt_review_not_in_single._merge == 'right_only']
print(len(nyt_review_not_in_single.author_1.unique()))
nyt_review_not_in_single.head()

61


,author_x,birth,death,race_eth,education,institution,year,week,rank,title_id,title,author_y,problem,author_alt,author_1,author_2,_merge
55,NaN,NaN,NaN,NaN,NaN,NaN,1995.0,1995-12-31,14.0,6336.0,THE SMALL ONE,Alex Walsh. Illustrated by Jesse Clay,True,Alex Walsh,Alex Walsh,NaN,right_only
92,NaN,NaN,NaN,NaN,NaN,NaN,2008.0,2008-03-02,15.0,974.0,CELEBUTANTES,Amanda Goldberg and Ruthanna Khalighi Hopper,True,Amanda Goldberg and Ruthanna Khalighi Hopper,Amanda Goldberg,Ruthanna Khalighi Hopper,right_only
99,NaN,NaN,NaN,NaN,NaN,NaN,2011.0,2011-01-23,9.0,6836.0,THREE SECONDS,Anders Roslund and Borge Hellstrom,True,Anders Roslund and Borge Hellstrom,Anders Roslund,Borge Hellstrom,right_only
100,NaN,NaN,NaN,NaN,NaN,NaN,2011.0,2011-01-30,8.0,6836.0,THREE SECONDS,Anders Roslund and Borge Hellstrom,True,Anders Roslund and Borge Hellstrom,Anders Roslund,Borge Hellstrom,right_only
101,NaN,NaN,NaN,NaN,NaN,NaN,2011.0,2011-02-06,15.0,6836.0,THREE SECONDS,Anders Roslund and Borge Hellstrom,True,Anders Roslund and Borge Hellstrom,Anders Roslund,Borge Hellstrom,right_only


In [44]:
#write to csv
nyt_review_not_in_single.to_csv('nyt_reviewed_authors.csv', index=False)

## Combine the single and reviewed author demographics 

In [57]:
demo_single = pd.read_csv("nyt_single_author_demographics.csv")
demo_single.rename(columns = {"Unnamed: 0":"author"}, inplace=True)

demo_review = pd.read_csv("nyt_reviewed_author_demographics.csv")
demo_review.drop("Unnamed: 0", axis = 1, inplace=True)


print(demo_single.head())
print(demo_review.head())

             author birth  death    race_eth           education  \
0   Warwick Deeping  1877   1950     English                None   
1  Mazo de la Roche  1879   1961    Canadian                None   
2     Pearl S. Buck  1892   1973   Caucasian               Ph.D.   
3      Willa Cather  1873   1947    American   Bachelor's Degree   
4   Rafael Sabatini  1875   1950     Italian                None   

                       institution  
0                             None  
1                             None  
2               Cornell University  
3   University of Nebraska-Lincoln  
4                             None  
                 author birth  death           race_eth           education  \
0            Alex Walsh   NaN   None               None                None   
1       Amanda Goldberg  1970    N/A           American   Bachelor's Degree   
2        Anders Roslund  1965    N/A            Swedish   Bachelor's Degree   
3     Armin L. Robinson  1953    N/A   African Americ

In [71]:
demographics = pd.concat([demo_single, demo_review])
print(len(demographics))
len(demographics.author.unique())

2081


2081

In [174]:
#should've fixed this earlier
demographics = demographics[demographics.author != "various authors"]
len(demographics.author.unique())

2080

## Clean the birthplace information

In [80]:
birth_single = pd.read_csv("nyt_single_author_birthplace.csv")
birth_single.rename(columns = {"Unnamed: 0":"author"}, inplace=True)

birth_review = pd.read_csv("nyt_reviewed_author_birthplace.csv")
birth_review.drop("Unnamed: 0", axis = 1, inplace=True)


print(birth_single.head())
print(birth_review.head())

             author                        birthplace
0   Warwick Deeping           Ramsgate, Kent, England
1  Mazo de la Roche        Newmarket, Ontario, Canada
2     Pearl S. Buck                Hillsboro, WV, USA
3      Willa Cather  Back Creek Valley, Virginia, USA
4   Rafael Sabatini              Fermo, Marche, Italy
                 author            birthplace
0            Alex Walsh                   NaN
1       Amanda Goldberg  Los Angeles, CA, USA
2        Anders Roslund     Stockholm, Sweden
3     Armin L. Robinson                   NaN
4  Arnaud de Borchgrave     Brussels, Belgium


In [269]:
birth = pd.concat([birth_single, birth_review])
print(len(birth))
len(birth.author.unique())

2025


2025

In [271]:
#should've fixed this earlier
birth = birth[birth.author != "various authors"]

In [273]:
birth['temp'] = birth.birthplace.str.split(',')
birth.head()

,author,birthplace,temp
0,Warwick Deeping,"Ramsgate, Kent, England","[Ramsgate, Kent, England]"
1,Mazo de la Roche,"Newmarket, Ontario, Canada","[Newmarket, Ontario, Canada]"
2,Pearl S. Buck,"Hillsboro, WV, USA","[Hillsboro, WV, USA]"
3,Willa Cather,"Back Creek Valley, Virginia, USA","[Back Creek Valley, Virginia, USA]"
4,Rafael Sabatini,"Fermo, Marche, Italy","[Fermo, Marche, Italy]"


In [275]:
#birthplace['list_len'] = birthplace.temp.apply(len)
print(birth.temp.iloc[0])
print(birth.temp.iloc[0][-1])
type(birth.temp.iloc[0])


['Ramsgate', ' Kent', ' England']
 England


list

In [277]:
def get_country(temp):
    if isinstance(temp, list) and temp:
        return temp[-1] 
    else: None


birth['country'] = birth.temp.apply(get_country)

In [279]:
birth.country.unique()

array([' England', ' Canada', ' USA', ' Italy', ' France', ' UK',
       ' Ireland', ' Austria', ' Scotland', None, ' New Zealand',
       ' Germany', ' Denmark', ' None', ' Austria-Hungary',
       ' South Africa', ' Czech Republic', ' Switzerland',
       ' Austria-Hungary (now Czech Republic)', ' Slovenia',
       ' British Guiana', ' Spain', ' Hungary', ' Norway', ' Finland',
       ' Iceland', ' China', ' Egypt', ' India', ' Poland', ' Russia',
       ' Reino Unido', ' Australia', ' Ukraine', ' Iceland.', ' Romania',
       ' Algeria', ' Northern Ireland', ' Sweden', ' Netherlands',
       ' Belgium', ' USA.', ' Greece', ' Wales', ' Japan',
       ' Russian Empire (now Ukraïne)', ' Indonesia', ' Israel',
       ' United Kingdom', ' Brazil', ' Poland)', ' South Korea',
       ' Turkey', ' Colombia', ' Iran', ' Kenya', ' Zambia', ' Peru',
       ' Vietnam', ' Panama', ' Trinidad and Tobago', ' Mexico',
       ' Sri Lanka', ' Zimbabwe', ' Haiti', ' Lebanon', ' Pakistan',
       ' Afg

In [281]:
birth.country = birth.country.replace(r" Poland\)", "Poland", regex = True)
birth.country = birth.country.replace(r"Austria-Hungary \(now Czech Republic\)", "Austria-Hungary", regex = True)
birth.country = birth.country.replace(r"Russian Empire \(now Ukraïne\)", "Ukraine", regex = True)
birth.country = birth.country.replace(r"Reino Unido|UK", "United Kingdom", regex = True)

birth.country = birth.country.replace(r"\.", "", regex = True)

birth.country = birth.country.replace(r"^\s+", "", regex = True)

In [283]:
birth.country.unique()

array(['England', 'Canada', 'USA', 'Italy', 'France', 'United Kingdom',
       'Ireland', 'Austria', 'Scotland', None, 'New Zealand', 'Germany',
       'Denmark', 'None', 'Austria-Hungary', 'South Africa',
       'Czech Republic', 'Switzerland', 'Slovenia', 'British Guiana',
       'Spain', 'Hungary', 'Norway', 'Finland', 'Iceland', 'China',
       'Egypt', 'India', 'Poland', 'Russia', 'Australia', 'Ukraine',
       'Romania', 'Algeria', 'Northern Ireland', 'Sweden', 'Netherlands',
       'Belgium', 'Greece', 'Wales', 'Japan', 'Indonesia', 'Israel',
       'Brazil', 'South Korea', 'Turkey', 'Colombia', 'Iran', 'Kenya',
       'Zambia', 'Peru', 'Vietnam', 'Panama', 'Trinidad and Tobago',
       'Mexico', 'Sri Lanka', 'Zimbabwe', 'Haiti', 'Lebanon', 'Pakistan',
       'Afghanistan', 'Dominican Republic', 'Serbia', 'Ghana', 'Cyprus',
       'Jamaica', 'Malaysia', 'Nigeria'], dtype=object)

In [285]:
def get_state(temp):
    if isinstance(temp, list) and temp:
        if temp[-1] == " USA":
            return temp[-2] 
    else: None


birth['state'] = birth.temp.apply(get_state)
birth.state.unique()

array([None, ' WV', ' Virginia', ' MI', ' CT', ' IL', ' IN', ' CO', ' PA',
       ' NY', ' SC', ' VT', ' Ohio', ' MS', ' VA', ' Massachusetts',
       ' MN', ' OH', ' D.C.', ' ME', ' IA', ' NJ', ' WY', ' MD', ' MA',
       ' KS', ' GA', ' CA', ' WI', ' NC', ' KY', ' OR', ' DE', ' MO',
       ' MT', ' AL', ' Nebraska', ' LA', ' ID', ' Mississippi', ' TN',
       ' WA', ' NH', ' New Jersey', ' Texas', ' ND', ' Kentucky', ' UT',
       ' AR', ' TX', ' RI', ' NV', '  New York', ' South Carolina', ' OK',
       ' Colorado', ' Arkansas', 'Maine', ' FL', ' NM', ' NE', ' AZ',
       ' Kansas', ' New York', ' HI', ' Oklahoma', ' Maryland', ' Iowa',
       ' Oregon', 'Pennsylvania', ' California', ' Pennsylvania',
       ' Connecticut', 'Nebraska', 'Connecticut', 'Alabama', ' Arizona',
       ' Alabama', 'Massachusetts', 'Illinois', 'New Jersey', ' AK',
       'Minnesota', 'North Carolina', 'California', ' None', 'Montana'],
      dtype=object)

In [289]:
birth.state = birth.state.replace(r"^\s+", "", regex = True)
birth.state = birth.state.replace(r"Virginia", "VA", regex = True)
birth.state = birth.state.replace(r"Massachusetts", "MA", regex = True)
birth.state = birth.state.replace(r"Nebraska|Omaha", "NE", regex = True)
birth.state = birth.state.replace(r"Mississippi", "MS", regex = True)
birth.state = birth.state.replace(r"New Jersey", "NJ", regex = True)
birth.state = birth.state.replace(r"Arkansas", "AK", regex = True)
birth.state = birth.state.replace(r"New York", "NY", regex = True)
#birth.state = birth.state.replace(r"New York|Brooklyn|New York City|Queens", "NY", regex = True)
birth.state = birth.state.replace(r"Texas", "TX", regex = True)
birth.state = birth.state.replace(r"Kentucky", "KY", regex = True)
birth.state = birth.state.replace(r"South Carolina", "SC", regex = True)
birth.state = birth.state.replace(r"Colorado", "CO", regex = True)
birth.state = birth.state.replace(r"Kansas", "KS", regex = True)
birth.state = birth.state.replace(r"Oklahoma", "OK", regex = True)
birth.state = birth.state.replace(r"Maryland", "MD", regex = True)
birth.state = birth.state.replace(r"Iowa", "IA", regex = True)
birth.state = birth.state.replace(r"Oregon", "OR", regex = True)
birth.state = birth.state.replace(r"California", "CA", regex = True)
birth.state = birth.state.replace(r"Pennsylvania", "PA", regex = True)
birth.state = birth.state.replace(r"Connecticut", "CT", regex = True)
birth.state = birth.state.replace(r"Alabama", "AL", regex = True)
birth.state = birth.state.replace(r"Ohio", "OH", regex = True)
birth.state = birth.state.replace(r"Arizona", "AZ", regex = True)
birth.state = birth.state.replace(r"Illinois", "IL", regex = True)
birth.state = birth.state.replace(r"Maine", "ME", regex = True)
birth.state = birth.state.replace(r"Minnesota", "MN", regex = True)
birth.state = birth.state.replace(r"North Carolina", "NC", regex = True)
birth.state = birth.state.replace(r"Montana", "MT", regex = True)
birth.state.unique()

array([None, 'WV', 'VA', 'MI', 'CT', 'IL', 'IN', 'CO', 'PA', 'NY', 'SC',
       'VT', 'OH', 'MS', 'MA', 'MN', 'D.C.', 'ME', 'IA', 'NJ', 'WY', 'MD',
       'KS', 'GA', 'CA', 'WI', 'NC', 'KY', 'OR', 'DE', 'MO', 'MT', 'AL',
       'NE', 'LA', 'ID', 'TN', 'WA', 'NH', 'TX', 'ND', 'UT', 'AR', 'RI',
       'NV', 'OK', 'AK', 'FL', 'NM', 'AZ', 'HI', 'None'], dtype=object)

In [321]:
def get_city(temp):
    if isinstance(temp, list) and temp:
        return temp[0] 
    else: None


birth['city'] = birth.temp.apply(get_city)

In [293]:
print(birth.head())
print(demographics.head())

             author                        birthplace  \
0   Warwick Deeping           Ramsgate, Kent, England   
1  Mazo de la Roche        Newmarket, Ontario, Canada   
2     Pearl S. Buck                Hillsboro, WV, USA   
3      Willa Cather  Back Creek Valley, Virginia, USA   
4   Rafael Sabatini              Fermo, Marche, Italy   

                                   temp  country state               city  
0           [Ramsgate,  Kent,  England]  England  None           Ramsgate  
1        [Newmarket,  Ontario,  Canada]   Canada  None          Newmarket  
2                [Hillsboro,  WV,  USA]      USA    WV          Hillsboro  
3  [Back Creek Valley,  Virginia,  USA]      USA    VA  Back Creek Valley  
4              [Fermo,  Marche,  Italy]    Italy  None              Fermo  
             author birth  death    race_eth           education  \
0   Warwick Deeping  1877   1950     English                None   
1  Mazo de la Roche  1879   1961    Canadian                None 

## Combine

In [893]:
auth_demo_birth = demographics.merge(birth, how='left', left_on = "author", right_on = "author", indicator = True)
auth_demo_birth = auth_demo_birth.replace(r"^\s+", "", regex = True)
auth_demo_birth[auth_demo_birth._merge !="both"].head()

,author,birth,death,race_eth,education,institution,birthplace,temp,country,state,city,_merge
2024,Arthur Quiller-Couch,1863,1944,British,None,None,NaN,NaN,NaN,NaN,NaN,left_only
2025,Barry Lopez,1945,N/A,Caucasian,Bachelor's Degree,University of Oregon,NaN,NaN,NaN,NaN,NaN,left_only
2026,Benedict Freedman,1904,1987,White,Bachelor's Degree,University of California,NaN,NaN,NaN,NaN,NaN,left_only
2027,Bethenny Frankel,1970,N/A,Caucasian,Bachelor's Degree,New York University,NaN,NaN,NaN,NaN,NaN,left_only
2028,Bill Adler,1942,N/A,White,Bachelor's Degree,City College of New York,NaN,NaN,NaN,NaN,NaN,left_only


In [895]:
auth_demo_birth.birth.unique()

array(['1877', '1879', '1892', '1873', '1875', '1867', '1882', '1885',
       '1888', '1901', '1886', '1881', '1866', '1884', '1591', '1869',
       '1876', '1894', '1902', '1896', '1895', '1947', '1880', '1893',
       '1890', '1887', '1870', '1874', '1903', '1889', '1904', nan,
       '1911', '1899', '1864', '1943', '1878', '1865', '1905', '1898',
       '1900', '1763', '1914', '1908', '1916', '1897', '1891', '1871',
       '1910', '1906', '1936', '1863', '1907', '1945', '1918', '1944',
       '1946', '1757', '1970', '1883', '1909', '1981', '1912', '1992',
       '1971', '1941', '1913', '1846', '1860', '1919', '1862', '1940',
       '1915', '1924', '1859', '1929', '1980', '1977', '1952', '1935',
       '1926', '1985', '1797', '1988', '1939', '1925', '1928', '1922',
       '1923', '1850', '1917', '1921', '1858', '1793', '1951', '1835',
       '1934', '1920', '1872', '1958', '1927', '1972', '1932', '1931',
       '1938', '1959', '1933', '1956', '1990', '1930', '1856', '1942',
       '1

In [897]:
auth_demo_birth[auth_demo_birth.author == 'Douglas Preston']

,author,birth,death,race_eth,education,institution,birthplace,temp,country,state,city,_merge
1544,Douglas Preston,Douglas Preston,1956,N/A,Caucasian,Bachelor's Degree,"Cambridge, MA, USA","[Cambridge, MA, USA]",USA,MA,Cambridge,both


In [899]:
#fix Douglas Preston
auth_demo_birth.birth = auth_demo_birth.birth.replace(r"Douglas Preston", "1956", regex = True)
auth_demo_birth.loc[1544, 'death'] = "N/A"
auth_demo_birth.loc[1544, 'race_eth'] = "Caucasian"
auth_demo_birth.loc[1544, 'education'] = "Bachelor's Degree"
auth_demo_birth.loc[1544, 'institution'] = "Pomona College"

In [921]:
auth_demo_birth[auth_demo_birth.institution =='Tulane University  \nWilliam R. Forstchen: 1950']

,author,birth,death,race_eth,education,institution,birthplace,temp,country,state,city,_merge
2068,"Newt Gingrich, William R. Forstchen",1943,N/A,White,Graduate,Tulane University \nWilliam R. Forstchen: 1950,NaN,NaN,NaN,NaN,NaN,left_only


In [925]:
#Fix JA Vance
auth_demo_birth.loc[1694, 'birth'] = "1944"
# Drop Newt Gingrich + Willian Forstchen, reducing to just Newt (2067)
auth_demo_birth.drop(2068, inplace = True)

In [903]:
auth_demo_birth.birth.unique()

array(['1877', '1879', '1892', '1873', '1875', '1867', '1882', '1885',
       '1888', '1901', '1886', '1881', '1866', '1884', '1591', '1869',
       '1876', '1894', '1902', '1896', '1895', '1947', '1880', '1893',
       '1890', '1887', '1870', '1874', '1903', '1889', '1904', nan,
       '1911', '1899', '1864', '1943', '1878', '1865', '1905', '1898',
       '1900', '1763', '1914', '1908', '1916', '1897', '1891', '1871',
       '1910', '1906', '1936', '1863', '1907', '1945', '1918', '1944',
       '1946', '1757', '1970', '1883', '1909', '1981', '1912', '1992',
       '1971', '1941', '1913', '1846', '1860', '1919', '1862', '1940',
       '1915', '1924', '1859', '1929', '1980', '1977', '1952', '1935',
       '1926', '1985', '1797', '1988', '1939', '1925', '1928', '1922',
       '1923', '1850', '1917', '1921', '1858', '1793', '1951', '1835',
       '1934', '1920', '1872', '1958', '1927', '1972', '1932', '1931',
       '1938', '1959', '1933', '1956', '1990', '1930', '1856', '1942',
       '1

In [905]:
auth_demo_birth.death.unique()

array(['1950', '1961', '1973', '1947', '1933', '1956', '1968', '1965',
       '1963', '1946', '1960', '1966', '1954', '1674', '1957', '1958',
       '1959', '1970', 'N/A', '1981', '1972', '1978', '1953', '1984',
       '1944', '1974', '1945', '1941', '1977', '1934', '1990', '1951',
       '1939', '1985', '1991', 'None', '1949', '1962', '1967', '1976',
       '2010', '1942', '1940', '1975', '1943', '1992', '1955', '1964',
       '1855', '1989', '2011', '1938', '1952', '1987', '1935', '1948',
       '2003', '2018', '1983', '1998', '1996', '2008', '1993', '1986',
       '1980', '1827', '2002', '1994', '2009', '2019', '2015', '2001',
       '1915', '1982', '1997', '2005', '1926', '1912', '1971', '1979',
       '1988', '1995', '1969', '1916', '199 Hodgins', '195 Lewisohn',
       '1869', '2012', '2007', '2016', '1999', '1932', '2006', '1860',
       '1936', '2013', '1874', '2017', '2000', '2004', '1931', '1902',
       '2014', '1930', '2021', '2020', '2022', '1822', '1859', '2023',
       '

In [907]:
#fix Hodgins
auth_demo_birth.loc[371, 'death'] = "1971"
#Fix Lewisohn
auth_demo_birth.loc[388, 'death'] = "1955"

In [913]:
auth_demo_birth.education.unique()

array(['None', 'Graduate', 'Undergrad ', 'N/A', 'High School', 'Medical'],
      dtype=object)

In [911]:
auth_demo_birth.education = auth_demo_birth.education.replace(r"Ph\.D\.|Doctorate|Doctoral Degree|Doctor of Sacred Theology|PhD|MFA|Juris Doctor|J/.D/. Degree|J.D. Degree|J.D.", "Graduate", regex = True)
auth_demo_birth.education = auth_demo_birth.education.replace(r"Bachelor’s Degree|Bachelor's Degree", "Undergrad ", regex = True)
auth_demo_birth.education = auth_demo_birth.education.replace(r"^Medical Degree$|^Doctor of Medicine$|^Doctor of Medicine \(MD\)$|^MD$", "Medical", regex = True)
auth_demo_birth.education = auth_demo_birth.education.replace(r"Degree in Modern Languages|Master's Degree|Master’s Degree|Graduate Degree|Masters", "Graduate", regex = True)
auth_demo_birth.education = auth_demo_birth.education.replace(r"Some college|High School Diploma", r"High School", regex = True)

In [915]:
auth_demo_birth.institution.unique()

array(['None', 'Cornell University', 'University of Nebraska-Lincoln',
       'New College', 'University of Wisconsin-Madison',
       'Harvard University', 'University of Chicago',
       'Birmingham University', 'Princeton University',
       'University of Colorado', 'Oxford University',
       'Columbia University', 'University of Vienna',
       'University of Oxford', 'University of the Pacific',
       'University of Glasgow', 'Ohio State University',
       'Barnard College', 'University of Pennsylvania',
       'University of Alabama', 'Smith College',
       'University of Cambridge', 'Newnham College',
       'University of Munich', 'University of Kansas', 'Yale University',
       'Balliol College', 'University of Florida',
       'University of California', 'University of Iowa',
       'University College London', 'University of Nebraska',
       'University of Toronto', 'University of Oslo', 'Vassar College',
       'University College Dublin', 'Washington College',
     

,author,birth,death,race_eth,education,institution,birthplace,temp,country,state,city,_merge
2068,"Newt Gingrich, William R. Forstchen",1943,N/A,White,Graduate,Tulane University \nWilliam R. Forstchen: 1950,NaN,NaN,NaN,NaN,NaN,left_only
